In [1]:
try:
    with open('i.html') as fp:
        text = fp.read()
except UnicodeDecodeError as exc:
    print(exc)

'utf-8' codec can't decode byte 0x80 in position 1721: invalid start byte


In [2]:
with open('i.html', 'rb') as fp:
    octets = fp.read()

for i in range(1715, 1725):
    code = octets[i]
    octet = octets[i:i+1]
    char = octet.decode('cp1252')
    print(f'{code:02x} {char!r}')

0a '\n'
0a '\n'
3c '<'
21 '!'
2d '-'
2d '-'
80 '€'
80 '€'
80 '€'
80 '€'


In [3]:
with open('i.html', encoding='cp1252') as fp:
     text = fp.read()

In [4]:
import re
import html

line_re = [re.compile(regex) for regex in (
    r'(°?) ?(Six|Nine) (at|in) the beginning means:',
    r'(°?) ?(Six|Nine)  ?in t?h?e second place means:',
    r'(°?) ?(Six|Nine)  ?in the third place means[:.]',
    r'(°?) ?(Six|Nine) in the fourth place means:',
    r'(°?) ?(Six|Nine) in the fifth place means:',
    r'(°?) ?(Six|Nine) at the top means:',
)]

def capture_line(n, verse):
    hit = line_re[n-1].search(verse)
    if hit is not None:
        return hit.group(1), hit.group(2)

hexagram_start = re.compile(r'<a name="\d{1,2}"></a>')
title_parts = re.compile(r'(\d{1,2})\. ([^\n]+)')
chapters = hexagram_start.split(html.unescape(text))
for i, chapter in enumerate(chapters[1:], 1):
    verses = chapter.split('\n')
    hit = title_parts.search(verses[1])
    n, title = hit.groups(verses[1])
    title_wg, title_en = (s.strip() for s in title.split('/'))
    assert i == int(n)
    print(f'{n:2}\t{title_wg:10} {title_en}')
    line_matches = 0
    iter_verses = iter(verses[2:])
    while True:
        try:
            verse = next(iter_verses)
        except StopIteration:
            break
        for n in range(1, 7):
            hit = capture_line(n, verse)
            if hit is not None:
                governing, value = hit
                line = 6 if value == 'Six' else 9
                print('\t', n, governing)
                line_matches += 1
                verse = next(iter_verses)
                while verse.startswith('\t'):
                    print('\t\t', verse)
                    verse = next(iter_verses)
                
    if line_matches != 6:
        raise ValueError('!!! missing line text')

1 	Ch'ien     The Creative
	 1 
		 	Hidden dragon. Do not act.
	 2 
		 	Dragon appearing in the field.
		 	It furthers one to see the great man.
	 3 
		 	All day long the superior man is creatively active.
		 	At nightfall his mind is still beset with cares.
		 	Danger. No blame.
	 4 
		 	Wavering flight over the depths.
		 	No blame.
	 5 °
		 	Flying dragon in the heavens.
		 	It furthers one to see the great man.
	 6 
		 	Arrogant dragon will have cause to repent.
2 	K'un       The Receptive
	 1 
		 	When there is hoarfrost underfoot,
		 	Solid ice is not far off.
	 2 °
		 	Straight, square, great.
		 	Without purpose,
		 	Yet nothing remains unfurthered.
	 3 
		 	Hidden lines.
		 	One is able to remain persevering.
		 	If by chance you are in the service of a king,
		 	Seek not works, but bring to completion.
	 4 
		 	A tied-up sack. No blame, no praise.
	 5 
		 	A yellow lower garment brings supreme good fortune.
	 6 
		 	Dragons fight in the meadow.
		 	Their blood is black and ye

21	Shih Ho    Biting Through
	 1 
		 	His feet are fastened in the stocks,
		 	So that his toes disappear.
		 	No blame.
	 2 
		 	Bites through tender meat,
		 	So that his nose disappears.
		 	No blame.
	 3 
		 	Bites on old dried meat 
		 	And strikes on something poisonous.
		 	Slight humiliation.  No blame.
	 4 
		 	Bites on dried gristly meat.
		 	Receives metal arrows.
		 	It furthers one to be mindful of difficulties
		 	And to be persevering.
		 	Good fortune. 
	 5 °
		 	Bites on dried lean meat.
		 	Receives yellow gold.
		 	Perseveringly aware of danger.
		 	No blame.
	 6 
		 	His neck is fastened in the wooden cangue,
		 	So that his ears disappear.
		 	Misfortune.
22	Pi         Grace
	 1 
		 	He lends grace to his toes, leaves the carriage, and walks.
	 2 °
		 	Lends grace to the beard on his chin.
	 3 
		 	Graceful and moist.
		 	Constant perseverance brings good fortune.
	 4 
		 	Grace or simplicity?
		 	A white horse comes as if on wings.
		 	He is not a robber,
		 	He w

		 	Perseverance furthers.
	 6 °
		 	The ting has rings of jade.
		 	Great good fortune.
		 	Nothing that would not act to further.
51	Chên       The Arousing (Shock, Thunder)
	 1 °
		 	Shock comes-oh, oh!
		 	Then follow laughing words-ha, ha!
		 	Good fortune.
	 2 
		 	Shock comes bringing danger.
		 	A hundred thousand times
		 	You lose your treasures
		 	And must climb the nine hills.
		 	Do not go in pursuit of them.
		 	After seven days you will get them back again.
	 3 
		 	Shock comes and makes one distraught.
		 	If shock spurs to action
		 	One remains free of misfortune.
		 	
	 4 
		 	Shock is mired.
	 5 
		 	Shock goes hither and thither.
		 	Danger.
		 	However, nothing at all is lost.
		 	Yet there are things to be done.
	 6 
		 	Shock brings ruin and terrified gazing around.
		 	Going ahead brings misfortune.
		 	If it has not yet touched one's own body
		 	But has reached one's neighbor first,
		 	There is no blame.
		 	One's comrades have something to talk about.
52	K